In [10]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import os
import re
import json
from datetime import datetime, timedelta

In [12]:
state = 'Rajasthan'
mandi = 'Kota'
exp_no = 6
retrainFreq = 15
actualPricesFilePath = os.path.join(f'../Data/PlottingData/SOYABEAN/ARIMA_Interpolated_Data/', f'{state.upper()}_{mandi.upper()}_Price.csv')
recommendationFilePath = f'./EXP{exp_no}_CYCLIC_{state.upper()}_{mandi.upper()}/Recommendations/RetrainFreq{retrainFreq}Day/'
withErrorModelFilePath = os.path.join(recommendationFilePath, 'WithErrorModels')
withoutErrorModelFilePath = os.path.join(recommendationFilePath, 'WithoutErrorModels')
processedFilePath = os.path.join(recommendationFilePath, 'Processed')

### Renaming files

In [13]:
def renameFiles(path, pattern, rpattern):
    comp = re.compile(pattern)
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if os.path.isfile(full_path):
            match = comp.search(f)
            if not match :
                continue

            try:
                new_name = match.expand(rpattern)
                new_name = os.path.join(path, new_name)
            except re.error:
                continue
        
            if os.path.isfile(new_name):
                print('%s -> %s skipped' % (f, new_name))
            else:
                os.rename(full_path, new_name)


In [14]:
pattern, rpattern = r'^recommend_with_errormodel_(\d{0,4}).csv$', r"Day_\1.csv"
renameFiles(withErrorModelFilePath, pattern, rpattern)

In [15]:
pattern, rpattern = r'^recommend_without_errormodel_(\d{0,4}).csv$', r"Day_\1.csv"
renameFiles(withoutErrorModelFilePath, pattern, rpattern)

### Process Recommendations

In [16]:
def processFile(recommendationsPath, actualPath):
    actual_df = pd.read_csv(actualPath, index_col=['DATE'])
    res = pd.DataFrame()
    all_files = os.listdir(recommendationsPath)
    all_files.sort()
    for fileName in all_files:
        path = os.path.join(recommendationsPath, fileName)
        r_df = pd.read_csv(path, index_col=['DATE'])
        a_df = actual_df[actual_df.index.isin(r_df.index)]
        
        curr_day = {}
        curr_day['DATE'] = a_df.index[0]
        curr_day['ACTUAL_PRICE'] = a_df.iloc[0]['PRICE'] # Actual price of day_0
        curr_day['ACTUAL_MAX'] = a_df[['PRICE']].max().iat[0]

        # Without Prospect Theory
        curr_day['MEAN_PRICE'] = r_df.iloc[0]['MEAN_PRICE'] # Mean price of day_0
        curr_day['MAX_MEAN_PRICE'] = r_df[['MEAN_PRICE']].max().iat[0] # Max mean price from day_0..day_29
        curr_day['MAX_MEAN_PRICE_RECOMMEND_DATE'] = r_df[['MEAN_PRICE']].idxmax().iat[0] # Max mean price date from day_0..day_29, recommended date by max value forecast method
        curr_day['MAX_MEAN_RECOMMEND_DAY_PRICE'] = a_df[a_df.index == curr_day['MAX_MEAN_PRICE_RECOMMEND_DATE']]['PRICE'].item() # Actual price on the recommended date by max value forecast method
        # With Prospect Theory
        curr_day['PROSPECT_RECOMMEND_DATE'] = r_df[['PREDICTED']].idxmax().iat[0] # recommended date by prospect theory method
        curr_day['PROSPECT_RECOMMEND_DAY_PRICE'] = a_df.loc[curr_day['PROSPECT_RECOMMEND_DATE']]['PRICE'].item() # Actual price on the recommended date 
        
        curr_df = pd.DataFrame([curr_day])
        res = pd.concat([res, curr_df])
    return res


In [17]:
if not os.path.exists(processedFilePath):
    os.makedirs(processedFilePath)

In [18]:
def seperateForecastMethods(df):
    # Mean value forecast method
    mean_price_df = df[['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'MAX_MEAN_PRICE_RECOMMEND_DATE', 'MAX_MEAN_RECOMMEND_DAY_PRICE']].copy()
    mean_price_df.rename(columns={'MAX_MEAN_PRICE_RECOMMEND_DATE' : 'RECOMMEND_DATE', 'MAX_MEAN_RECOMMEND_DAY_PRICE' : 'RECOMMEND_DAY_PRICE'} , inplace=True)
    # Prospect theory method
    prospect_df = df[['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'PROSPECT_RECOMMEND_DATE', 'PROSPECT_RECOMMEND_DAY_PRICE']].copy()
    prospect_df.rename(columns={'PROSPECT_RECOMMEND_DATE' : 'RECOMMEND_DATE', 'PROSPECT_RECOMMEND_DAY_PRICE' : 'RECOMMEND_DAY_PRICE'} , inplace=True)
    return mean_price_df, prospect_df

In [19]:
# Process files with error models
with_error_df = processFile(withErrorModelFilePath, actualPricesFilePath)
with_error_mean_price_df, with_error_prospect_df = seperateForecastMethods(with_error_df)
with_error_mean_price_df.to_csv(os.path.join(processedFilePath, 'with_error_mean_price.csv'), index=False)
with_error_prospect_df.to_csv(os.path.join(processedFilePath, 'with_error_prospect.csv'), index=False)

In [20]:
# Process file without error models
without_error_df = processFile(withoutErrorModelFilePath, actualPricesFilePath)
without_error_mean_price_df, without_error_prospect_df = seperateForecastMethods(without_error_df)
without_error_mean_price_df.to_csv(os.path.join(processedFilePath, 'without_error_mean_price.csv'), index=False)
without_error_prospect_df.to_csv(os.path.join(processedFilePath, 'without_error_prospect.csv'), index=False)

### Compute Metrics

In [11]:
# p_df - ['DATE', 'ACTUAL_PRICE', 'ACTUAL_MAX', 'RECOMMEND_DATE', 'RECOMMEND_DAY_PRICE']
def computeMetrics(p_df):
    ans = {}
    
    n = p_df.shape[0]
    diff = 0
    p_df['DATE'] = pd.to_datetime(p_df['DATE'])
    p_df['RECOMMEND_DATE'] = pd.to_datetime(p_df['RECOMMEND_DATE'])

    for i in range(n-1):
        j = i + 1
        diff += abs((p_df.iloc[i]['RECOMMEND_DATE'] - p_df.iloc[j]['RECOMMEND_DATE']).days)
    
    count = 0
    threshold = 2
    for i in range(n-1):
        j = i + 1
        if abs((p_df.iloc[i]['RECOMMEND_DATE'] - p_df.iloc[j]['RECOMMEND_DATE']).days) <= threshold:
            count += 1
    
    ans['VOR'] = diff / (n-1)
    ans['PCR'] = count*100 / (n-1)
    ans['PAP'] = sum(p_df['ACTUAL_PRICE'] <= p_df['RECOMMEND_DAY_PRICE']) * 100 / n
    ans['NG'] = sum(p_df['RECOMMEND_DAY_PRICE'] - p_df['ACTUAL_PRICE']) / n # Actual price of day 0
    ans['RMSE_ORACLE'] = (sum((p_df['ACTUAL_MAX'] - p_df['RECOMMEND_DAY_PRICE']) ** 2) / n) ** 0.5  
    ans['NG_ORACLE'] = sum(p_df['ACTUAL_MAX'] - p_df['ACTUAL_PRICE']) / n

    return ans

In [12]:
def generateMetrics(start_date, end_date):
    data = {
        'Model': [],
        'Recommendation Method': [],
        'Metric': [],
        'Value': []
    }

    for fname in os.listdir(processedFilePath):
        if 'netgain' in fname or os.path.isdir(os.path.join(processedFilePath, fname)):
            continue
        p_df = pd.read_csv(os.path.join(processedFilePath, fname))
        p_df = p_df[(p_df['DATE'] >= start_date) & (p_df['DATE'] <= end_date)]
        model = 'ERROR' if 'with_error' in fname else 'WITHOUT_ERROR'
        recommendation_method = 'MAX_MEAN' if 'mean_price' in fname else 'PROSPECT'
        metrics = computeMetrics(p_df)
        data['Model'] += ([model]*len(metrics))
        data['Recommendation Method'] += ([recommendation_method]*len(metrics))
        data['Metric'] += (list(metrics.keys()))
        data['Value'] += (list(metrics.values()))

    metrics_df = pd.DataFrame(data)
    # print(metrics_df['Metric'])
    metrics_df = metrics_df.pivot(index='Metric', columns=['Model', 'Recommendation Method'], values='Value')
    return metrics_df

### Saving computed metrics seasonal and yearly

In [205]:
for year in range(2014, 2021):
    metricsPath = os.path.join(processedFilePath, 'metrics')
    if not os.path.exists(metricsPath):
        os.makedirs(metricsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    seasonal_df = generateMetrics(start_date, end_date)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    yearly_df = generateMetrics(start_date, end_date)
    seasonal_df.to_csv(os.path.join(metricsPath, f'metrics_seasonal_09_{year}_01_{year+1}.csv'))
    yearly_df.to_csv(os.path.join(metricsPath, f'metrics_yearly_{year}.csv'))

In [206]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
final_df = generateMetrics(start_date, end_date)
final_df.to_csv(os.path.join(os.path.join(processedFilePath, 'metrics'), f'metrics_2014_2020.csv'))

In [207]:
print(metricsPath)

./EXP24_RAJASTHAN_BHAWANI/Recommendations/RetrainFreq15Day/Processed/metrics


### Plotting and saving plots seasonal and yearly

In [26]:
def plotActualForecast(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index)
    final_df = pd.DataFrame()
    for idx in range(start_idx, end_idx, 84):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index)
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        final_df = pd.concat([ final_df, p_df[['PRICE', 'MEAN_PRICE']] ])
    final_df = final_df[final_df.index <= pd.to_datetime(end_date)]
    fig, ax = plt.subplots(figsize=(15, 8))
    ax.plot(final_df.index, final_df['PRICE'], label='Actual', color='green')
    ax.plot(final_df.index, final_df['MEAN_PRICE'], label='Forecasted', color='red')
    ax.vlines(x=pd.date_range(start=datetime.strptime(start_date, "%Y-%m-%d"), end=datetime.strptime(end_date, "%Y-%m-%d"), freq='84D'), ymin = min(final_df['MEAN_PRICE'].min(), final_df['PRICE'].min()) - 50, ymax = max(final_df['MEAN_PRICE'].max(), final_df['PRICE'].max()) + 50, colors='purple', ls='--', lw=2)
    # day_locater = mdates.DayLocator(interval=84)
    # day_formatter = mdates.DateFormatter("%b")
    # ax.xaxis.set_major_locator(day_locater)
    # ax.xaxis.set_major_formatter(month_formatter)
    ax.set_xlabel(f'Years')
    ax.set_ylabel('Price(in Rs)')
    ax.set_title(f'Soyabean prices from {start_date} to {end_date}')
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ax.legend()
    return fig

In [27]:
start_date, end_date = f'2014-01-01', f'2021-01-31'
fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(fig)

FileNotFoundError: [Errno 2] No such file or directory: './EXP3_CYCLIC_RAJASTHAN_KOTA/Recommendations/RetrainFreq15Day/WithErrorModels/Day_3342.csv'

In [209]:
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)    
    fig.savefig(os.path.join(plotsPath, f'plot_seasonal_09_{year}_01_{year+1}.png'), facecolor='w')
    plt.close(fig)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
    fig.savefig(os.path.join(plotsPath, f'plot_yearly_{year}.png'), facecolor='w')
    plt.close(fig)

In [210]:
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    fig.savefig(os.path.join(plotsPath, f'plot_seasonal_without_error_09_{year}_01_{year+1}.png'), facecolor='w')
    plt.close(fig)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    fig = plotActualForecast(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)
    fig.savefig(os.path.join(plotsPath, f'plot_yearly_without_error_{year}.png'), facecolor='w')
    plt.close(fig)

### Generate files with netgain computed on a daily basis

In [211]:
for fname in os.listdir(processedFilePath):
    if 'with' not in fname:
        continue
    p_df = pd.read_csv(os.path.join(processedFilePath, fname))
    p_df['NET_GAIN'] = p_df['RECOMMEND_DAY_PRICE'] - p_df['ACTUAL_PRICE']
    p_df['NET_GAIN_ORACLE'] = p_df['ACTUAL_MAX'] - p_df['ACTUAL_PRICE']
    p_df.to_csv(os.path.join(processedFilePath, f"netgain_{fname}"), index=False)

### Plotting netgain on a daily basis

In [212]:
def plotNetGain(start_date, end_date):
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    for fname in os.listdir(processedFilePath):
        if 'netgain' not in fname:
            continue
        p_df = pd.read_csv(os.path.join(processedFilePath, fname))
        p_df = p_df[(p_df['DATE'] >= start_date) & (p_df['DATE'] <= end_date)]
        model = 'with_error' if 'with_error' in fname else 'without_error'
        recommendation_method = 'max_mean' if 'mean_price' in fname else 'prospect'
        fig, ax = plt.subplots(figsize=(15, 8))
        ax.plot(p_df.index, p_df['NET_GAIN'], label='Net Gain', color='green')
        ax.plot(p_df.index, p_df['NET_GAIN_ORACLE'], label='Net Gain Oracle', color='red')
        month_locater = mdates.MonthLocator()
        month_formatter = mdates.DateFormatter("%b")
        ax.xaxis.set_major_locator(month_locater)
        ax.xaxis.set_major_formatter(month_formatter)
        ax.set_xlabel(f'Months')
        ax.set_ylabel('Price(in Rs)')
        ax.set_title(f'Net Gain and Net Gain Oracle from {start_date} to {end_date}')
        for tick in ax.get_xticklabels():
            tick.set_rotation(45)
        ax.legend()
        fig.savefig(os.path.join(plotsPath, f'netgain_{model}_{recommendation_method}_plot_{start_dt.month}_{start_dt.year}_{end_dt.month}_{end_dt.year}.png'), facecolor='w')
        plt.close(fig)

In [213]:
for year in range(2014, 2021):
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    plotNetGain(start_date, end_date)
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    plotNetGain(start_date, end_date)

### Computing RMSE

In [21]:
# RMSE over interval:
def rmse30DayWindow(df):
    mse = (df["PRICE"] - df["MEAN_PRICE"]) ** 2
    rmse = (mse.mean()) ** .5
    return rmse

def RMSE(df):
    l30, l1, lnormalized = [], [], []
    for i in range(0, len(df), 12):
        x30 = rmse30DayWindow(df[i:i + 12])
        l30.append(x30)
    return np.mean(l30)

In [57]:
def compute_weekly_trend(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index)
    fp_ap, fp_an, fn_ap, fn_an = 0, 0, 0, 0
    fp_ap_value, fp_an_value, fn_ap_value, fn_an_value = 0, 0, 0, 0
    for idx in range(start_idx, end_idx, 84):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index)
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        # t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        for i in range(1, p_df.shape[0]):
            a_diff = p_df.iloc[i]['PRICE'] - p_df.iloc[i-1]['PRICE']
            f_diff = p_df.iloc[i]['MEAN_PRICE'] - p_df.iloc[i-1]['MEAN_PRICE']
            abs_diff = abs(p_df.iloc[i]['MEAN_PRICE'] - p_df.iloc[i]['PRICE'])
            if a_diff < 0 and f_diff < 0:
                fn_an += 1
                fn_an_value += abs_diff
            elif a_diff >= 0 and f_diff >= 0:
                fp_ap += 1
                fp_ap_value += abs_diff
            elif a_diff < 0 and f_diff >= 0:
                fp_an += 1
                fp_an_value += abs_diff
            elif a_diff >= 0 and f_diff < 0:
                fn_ap += 1
                fn_ap_value += abs_diff

    count_list = [fp_ap, fn_an, fp_an, fn_ap]
    count_total = fp_ap + fp_an + fn_ap + fn_an
    
    value_list = [fp_ap_value, fn_an_value, fp_an_value, fn_ap_value]
    return [ count/count_total for count in count_list], [x/y for x, y in zip(value_list, count_list)]

In [58]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
trends = compute_weekly_trend(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(trends)

([0.2375366568914956, 0.25513196480938416, 0.2375366568914956, 0.2697947214076246], [438.4319122220469, 418.66838777157375, 411.95570915925504, 351.1219665793595])


In [48]:
def compute_weekly_trend_daily(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index)
    fp_ap, fp_an, fn_ap, fn_an = 0, 0, 0, 0
    for idx in range(start_idx, end_idx, 7):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index)
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        # t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        for i in range(1, p_df.shape[0]):
            a_diff = p_df.iloc[i]['PRICE'] - p_df.iloc[i-1]['PRICE']
            f_diff = p_df.iloc[i]['MEAN_PRICE'] - p_df.iloc[i-1]['MEAN_PRICE']
            if a_diff < 0 and f_diff < 0:
                fn_an = fn_an + 1
            elif a_diff >= 0 and f_diff >= 0:
                fp_ap = fp_ap + 1
            elif a_diff < 0 and f_diff >= 0:
                fp_an = fp_an + 1
            elif a_diff >= 0 and f_diff < 0:
                fn_ap = fn_ap + 1

    total = fp_ap + fp_an + fn_ap + fn_an
    l = [fp_ap, fn_an, fp_an, fn_ap]
    return [ x/total for x in l]

In [49]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
trends_daily = compute_weekly_trend_daily(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(trends_daily)

[0.2466467958271237, 0.22925981122702435, 0.2570789865871833, 0.26701440635866863]


In [52]:
def compute_weekly_mase(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index)
    final_df = pd.DataFrame()
    for idx in range(start_idx, end_idx, 84):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index)
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        # t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        final_df = pd.concat([final_df, p_df[['PRICE', 'MEAN_PRICE']]])
    final_df = final_df[final_df.index <= pd.to_datetime(end_date)]

    # mae
    final_df['DIFF'] = np.abs(final_df['MEAN_PRICE'] - final_df['PRICE'])
    mae = final_df['DIFF'].sum() / final_df.shape[0]
    
    # mae naive
    mae_naive = 0
    for i in range(1, final_df.shape[0]):
        mae_naive += abs(final_df.iloc[i]['MEAN_PRICE'] - final_df.iloc[i-1]['MEAN_PRICE'])
    mae_naive /= (final_df.shape[0] - 1)
    print(mae, mae_naive)
    return mae / mae_naive

In [53]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
mase = compute_weekly_mase(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(mase)

390.98684176722924 91.04198872003143
4.294577120558909


In [22]:
def compute_rmse(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index)
    final_df = pd.DataFrame()
    for idx in range(start_idx, end_idx, 84):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index)
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        # t_df = pd.merge(p_df, actual_df, left_index=True, right_index=True, how='inner')
        final_df = pd.concat([final_df, p_df[['PRICE', 'MEAN_PRICE']]])
    final_df = final_df[final_df.index <= pd.to_datetime(end_date)]
    n_df = final_df[['PRICE', 'MEAN_PRICE']].copy()
    rmse = RMSE(n_df)
    return rmse

In [26]:
start_date, end_date = f'2014-01-08', f'2020-12-31'
rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(rmse)

685.3200513974855


In [6]:
def compute_daily_rmse(start_date, end_date, actualPricesFilePath, recommendationPath):
    base_date = datetime.strptime("2006-01-01", "%Y-%m-%d")
    start_idx = (datetime.strptime(start_date, "%Y-%m-%d") - base_date).days
    end_idx = (datetime.strptime(end_date, "%Y-%m-%d") - base_date).days
    actual_df = pd.read_csv(actualPricesFilePath, index_col=['DATE'])
    actual_df.index = pd.to_datetime(actual_df.index, format='%Y-%m-%d')
    all_rmse = []
    for idx in range(start_idx, end_idx+1, 7):
        p_df = pd.read_csv(os.path.join(recommendationPath, f"Day_{idx}.csv"), index_col=['DATE'])
        p_df.index = pd.to_datetime(p_df.index, format='%Y-%m-%d')
        for dt in p_df.index:
            p_df.loc[dt, 'PRICE'] = actual_df.loc[dt:dt+timedelta(days=7), 'PRICE'].mean()
        t_rmse = ((p_df['PRICE'] - p_df['MEAN_PRICE']) ** 2).mean() ** .5
        all_rmse.append(t_rmse)
    return sum(all_rmse)/len(all_rmse)

In [8]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
daily_rmse = compute_daily_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)
print(daily_rmse)

FileNotFoundError: [Errno 2] No such file or directory: './EXP3_CYCLIC_RAJASTHAN_KOTA/Recommendations/RetrainFreq15Day/WithErrorModels/Day_3279.csv'

In [6]:
data = {'Type': [], 'Duration': [], 'Year': [], 'Rolling RMSE': [], 'Daily RMSE': []}

In [7]:
print("---- With Error Models ----")
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)

    data['Type'].append('Error')
    data['Duration'].append('Seasonal')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)

    print(f"Seasonal: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")
    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)    
    print(f"Yearly: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")
    
    data['Type'].append('Error')
    data['Duration'].append('Yearly')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)

---- With Error Models ----
Seasonal: Year: 2014, Rolling RMSE: 444.3806869667025, Daily RMSE: 474.7154874686746
Yearly: Year: 2014, Rolling RMSE: 412.2339714968385, Daily RMSE: 481.29415086047294
Seasonal: Year: 2015, Rolling RMSE: 138.48475406063838, Daily RMSE: 215.69863006714007
Yearly: Year: 2015, Rolling RMSE: 207.69260466586567, Daily RMSE: 250.86741093533354
Seasonal: Year: 2016, Rolling RMSE: 286.45998588958776, Daily RMSE: 301.3401813612108
Yearly: Year: 2016, Rolling RMSE: 202.02443725761233, Daily RMSE: 236.33690444962372
Seasonal: Year: 2017, Rolling RMSE: 309.12107890679914, Daily RMSE: 263.58189660391133
Yearly: Year: 2017, Rolling RMSE: 183.30493833304354, Daily RMSE: 191.77258253121246
Seasonal: Year: 2018, Rolling RMSE: 179.72840214326266, Daily RMSE: 197.93532232034843
Yearly: Year: 2018, Rolling RMSE: 233.98574715293367, Daily RMSE: 318.03166753226384
Seasonal: Year: 2019, Rolling RMSE: 271.9996273515591, Daily RMSE: 318.45072513886157
Yearly: Year: 2019, Rolling RM

In [8]:
print("---- Without Error Models ----")
for year in range(2014, 2021):
    plotsPath = os.path.join(processedFilePath, 'plots')
    if not os.path.exists(plotsPath):
        os.makedirs(plotsPath)
    start_date, end_date = f'{year}-09-01', f'{year+1}-01-31'
    # print(withoutErrorModelFilePath)
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    print(f"Seasonal: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")

    data['Type'].append('Without Error')
    data['Duration'].append('Seasonal')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)


    start_date, end_date = f'{year}-01-01', f'{year}-12-31'
    rmse, daily_rmse = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)    
    print(f"Yearly: Year: {year}, Rolling RMSE: {rmse}, Daily RMSE: {daily_rmse}")

    data['Type'].append('Without Error')
    data['Duration'].append('Yearly')
    data['Year'].append(year)
    data['Rolling RMSE'].append(rmse)
    data['Daily RMSE'].append(daily_rmse)

---- Without Error Models ----
Seasonal: Year: 2014, Rolling RMSE: 366.05327567567457, Daily RMSE: 399.8329189775516
Yearly: Year: 2014, Rolling RMSE: 381.8159590042818, Daily RMSE: 455.9772808008034
Seasonal: Year: 2015, Rolling RMSE: 151.10830312867958, Daily RMSE: 223.1352392468516
Yearly: Year: 2015, Rolling RMSE: 218.08424272088547, Daily RMSE: 264.0749045100583
Seasonal: Year: 2016, Rolling RMSE: 277.8201512477, Daily RMSE: 275.01414383593993
Yearly: Year: 2016, Rolling RMSE: 199.51492071999473, Daily RMSE: 239.49342808054425
Seasonal: Year: 2017, Rolling RMSE: 400.65269949139264, Daily RMSE: 342.06189935055437
Yearly: Year: 2017, Rolling RMSE: 248.53866813364792, Daily RMSE: 259.1271091555911
Seasonal: Year: 2018, Rolling RMSE: 185.68208472968422, Daily RMSE: 205.59954447130735
Yearly: Year: 2018, Rolling RMSE: 245.043471008195, Daily RMSE: 341.5293136342834
Seasonal: Year: 2019, Rolling RMSE: 258.01505026316823, Daily RMSE: 302.9170407622495
Yearly: Year: 2019, Rolling RMSE: 22

In [9]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
rmse_with, _ = compute_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)  
rmse_without, _ = compute_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)  
print("Rolling RMSE: ")
print(rmse_with, rmse_without)

Rolling RMSE: 
246.43504646057502 262.97454190913965


In [221]:
start_date, end_date = f'2014-01-01', f'2020-12-31'
rmse_with = compute_daily_rmse(start_date, end_date, actualPricesFilePath, withErrorModelFilePath)  
rmse_without = compute_daily_rmse(start_date, end_date, actualPricesFilePath, withoutErrorModelFilePath)  
print("Daily RMSE: ")
print(rmse_with, rmse_without)

Daily RMSE: 
197.02389599310456 214.83688303152766


In [222]:
rmse_df = pd.DataFrame(data)
rmse_df = pd.pivot(rmse_df, index='Year', columns=['Type', 'Duration'], values=['Rolling RMSE', 'Daily RMSE'])
print(rmse_df.head())
# print(processedFilePath)
rmse_df.to_csv(os.path.join(os.path.join(processedFilePath, 'metrics'), f'rmse_2014_2020.csv'))

         Rolling RMSE                                        Daily RMSE  \
Type            Error             Without Error                   Error   
Duration     Seasonal      Yearly      Seasonal      Yearly    Seasonal   
Year                                                                      
2014       179.669095  260.039985    173.207340  258.251118  253.906972   
2015       126.388888  152.377172    125.289158  182.901703  169.259621   
2016       174.538012  166.944566    205.019425  174.035210  226.799396   
2017       196.257473   93.212513    207.820647  121.279303  179.745426   
2018       165.349175  156.387631    184.456014  174.190620  172.751900   

                                                
Type                 Without Error              
Duration      Yearly      Seasonal      Yearly  
Year                                            
2014      309.219812    253.278105  319.632141  
2015      185.652953    177.510170  219.668743  
2016      204.734904    254.41